In [10]:
import pandas as pd
btc_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\whitepapers\btc.csv', index_col=None)
eth_df= pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\whitepapers\eth.csv',  index_col=None)
ada_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\whitepapers\ada.csv',  index_col=None)
sol_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\whitepapers\sol.csv',  index_col=None)

In [93]:
btc_with_sections_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\whitepapers\btc_with_sections.csv', index_col=None)

# clustering with TOP2Vec

In [86]:
pd.set_option('display.max_colwidth', None)
btc_with_sections_df.head(1)

,text,section
0,"A purely peer-to-peer version of electronic cash would allow online payments to be sent directly from one party to another without going through a financial institution. Digital signatures provide part of the solution, but the main benefits are lost if a trusted third party is still required to prevent double-spending. We propose a solution to the double-spending problem using a peer-to-peer network. The network timestamps transactions by hashing them into an ongoing chain of hash-based proof-of-work, forming a record that cannot be changed without redoing the proof-of-work. The longest chain not only serves as proof of the sequence of events witnessed, but proof that it came from the largest pool of CPU power. As long as a majority of CPU power is controlled by nodes that are not cooperating to attack the network, they'll generate the longest chain and outpace attackers. The network itself requires minimal structure. Messages are broadcast on a best effort basis, and nodes can leave and rejoin the network at will, accepting the longest proof-of-work chain as proof of what happened while they were gone.",Abstract


In [94]:
import nltk
btc_with_sections_df["sents"] =  btc_with_sections_df.apply(lambda x : nltk.tokenize.sent_tokenize(x["text"]), axis=1)

In [95]:
all_sents =  []
for i, row in btc_with_sections_df.iterrows():
    all_sents.extend(list(row["sents"]))

In [ ]:
all_sents

In [96]:
btc_all_text = pd.DataFrame({"sents": "", "text": "", "section": "all"}, index=[0])
btc_all_text.at[0, 'sents'] = all_sents
btc_with_sections_df = pd.concat([btc_with_sections_df, btc_all_text])

In [ ]:
btc_all_text

In [ ]:
btc_with_sections_df[btc_with_sections_df["section"] == 'all'].head(1)

In [113]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np 

embedder = SentenceTransformer('distiluse-base-multilingual-cased-v2')

def agglomerative_clustering(corpus, n_clusters=10):
    corpus_embeddings = embedder.encode(corpus)
    # Normalize the embeddings to unit length
    corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)
    clustering_model = AgglomerativeClustering(n_clusters=n_clusters, distance_threshold=None)#affinity='cosine', linkage='average', distance_threshold=0.4)
    clustering_model.fit_predict(corpus_embeddings)
    cluster_assignment = clustering_model.labels_
    clustered_sentences = {}
    for sentence_id, cluster_id in enumerate(cluster_assignment):
        if cluster_id not in clustered_sentences:
            clustered_sentences[cluster_id] = []
        clustered_sentences[cluster_id].append(corpus[sentence_id])
    
    df = pd.DataFrame({"cluster_id": clustered_sentences.keys(), "sentences": clustered_sentences.values()})
    return df
        
    

In [114]:
results = agglomerative_clustering(all_sents)

In [115]:
results.head(2)

,cluster_id,sentences
0,6,"[A purely peer-to-peer version of electronic cash would allow online payments to be sent directly from one party to another without going through a financial institution., Commerce on the Internet has come to rely almost exclusively on financial institutions serving as trusted third parties to process electronic payments., Completely non-reversible transactions are not really possible, since financial institutions cannot avoid mediating disputes., The cost of mediation increases transaction costs, limiting the minimum practical transaction size and cutting off the possibility for small casual transactions, and there is a broader cost in the loss of ability to make non-reversible payments for nonreversible services., These costs and payment uncertainties can be avoided in person by using physical currency, but no mechanism exists to make payments over a communications channel without a trusted party., Transactions that are computationally impractical to reverse would protect sellers from fraud, and routine escrow mechanisms could easily be implemented to protect buyers., A common solution is to introduce a trusted central authority, or mint, that checks every transaction for double spending., After each transaction, the coin must be returned to the mint to issue a new coin, and only coins issued directly from the mint are trusted not to be double-spent., The problem with this solution is that the fate of the entire money system depends on the company running the mint, with every transaction having to go through them, just like a bank., This adds an incentive for nodes to support the network, and provides\na way to initially distribute coins into circulation, since there is no central authority to issue them., The steady addition of a constant of amount of new coins is analogous to gold miners expending\nresources to add gold to circulation., Once a predetermined number of coins have entered\ncirculation, the incentive can transition entirely to transaction fees and be completely inflation\nfree., If a greedy attacker is able to\nassemble more CPU power than all the honest nodes, he would have to choose between using it\nto defraud people by stealing back his payments, or using it to generate new coins., He ought to\nfind it more profitable to play by the rules, such rules that favour him with more new coins than\neveryone else combined, than to undermine the system and the validity of his own wealth., Although it would be possible to handle coins individually, it would be unwieldy to make a separate transaction for every cent in a transfer., Even if this is accomplished, it does not throw the system open to arbitrary changes, such as creating value out of thin air or taking money that never belonged to the attacker., An attacker can only try to change one of his own transactions to take back money he recently spent., We assume the sender is an attacker who wants to make the recipient believe he paid him for a while, then switch it to pay back to himself after some time has passed.]"
1,1,"[Digital signatures provide part of the solution, but the main benefits are lost if a trusted third party is still required to prevent double-spending., The longest chain not only serves as proof of the sequence of events witnessed, but proof that it came from the largest pool of CPU power., We define an electronic coin as a chain of digital signatures., A payee can verify the signatures to verify the chain of ownership., The problem of course is the payee can't verify that one of the owners did not double-spend the coin., The proof-of-work also solves the problem of determining representation in majority decision making., If the majority were based on one-IP-address-one-vote, it could be subverted by anyone able to allocate many IPs., Proof-of-work is essentially one-CPU-one-vote., The majority decision is represented by the longest chain, which has the greatest proof-of-work effort invested in it., We started with t

In [104]:
len(cluster_assignment)

130

In [97]:
from top2vec import Top2Vec
def top_2_vec(row, results_df):
    corpus = list(row['sents'])
    print(len(corpus))
    model = Top2Vec(corpus,  embedding_model='distiluse-base-multilingual-cased-v2', hdbscan_args={"min_cluster_size":5}, min_count =5)
    topic_sizes, topic_nums = model.get_topic_sizes()
    topic_words, word_scores, topic_nums = model.get_topics(len(topic_nums))
    aspect_docs = []
    aspect_docs_scores = []
    sub_topic_ids = []
    topics_size = []
    topic_word = []
    word_score = []
    sections = []
    for t_s, t_n in zip(topic_sizes, topic_nums):
        sub_topic_ids.append(t_n)
        documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=t_n, num_docs=t_s)
        aspect_docs.append(documents)
        aspect_docs_scores.append(document_scores)
        topics_size.append(topic_sizes[t_n])
        topic_word.append(topic_words[t_n])
        word_score.append(word_scores[t_n])
        sections.append(row["section"])
    results_df = pd.concat([results_df, pd.DataFrame({ "section": sections, "aspect_id": sub_topic_ids, "aspect_docs": aspect_docs, "aspect_docs_scores": aspect_docs_scores, "topic_size": topics_size, "topic_words":topic_word, "word_scores": word_score})])
    return results_df

In [98]:
# good results were shown from paraphrase-multilingual-MiniLM-L12-v2, distiluse-base-multilingual-cased-v2

btc_with_sections_df = btc_with_sections_df[btc_with_sections_df["section"] == 'all']

results_df = pd.DataFrame({})
for index, row in btc_with_sections_df.iterrows():
    try:
        results_df = top_2_vec(row, results_df)
    except Exception as e:
        print(e)
        continue
    
results_df['centroid'] = results_df['aspect_docs'].apply(lambda row: row[0] )

2022-02-17 15:23:41,322 - top2vec - INFO - Pre-processing documents for training
C:\Users\Bilal\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
2022-02-17 15:23:41,338 - top2vec - INFO - Downloading distiluse-base-multilingual-cased-v2 model


130


2022-02-17 15:23:49,677 - top2vec - INFO - Creating joint document/word embedding
2022-02-17 15:23:51,334 - top2vec - INFO - Creating lower dimension embedding of documents
2022-02-17 15:23:53,648 - top2vec - INFO - Finding dense areas of documents
2022-02-17 15:23:53,656 - top2vec - INFO - Finding topics


In [116]:
# top 2 vec => all paper in one cluster
# agglomerative_clustering with sentence transformer, n_cluster = 10
results.to_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\whitepapers\agglomerative_clustering_btc.csv")

In [117]:
results.head(1)

,cluster_id,sentences
0,6,"[A purely peer-to-peer version of electronic cash would allow online payments to be sent directly from one party to another without going through a financial institution., Commerce on the Internet has come to rely almost exclusively on financial institutions serving as trusted third parties to process electronic payments., Completely non-reversible transactions are not really possible, since financial institutions cannot avoid mediating disputes., The cost of mediation increases transaction costs, limiting the minimum practical transaction size and cutting off the possibility for small casual transactions, and there is a broader cost in the loss of ability to make non-reversible payments for nonreversible services., These costs and payment uncertainties can be avoided in person by using physical currency, but no mechanism exists to make payments over a communications channel without a trusted party., Transactions that are computationally impractical to reverse would protect sellers from fraud, and routine escrow mechanisms could easily be implemented to protect buyers., A common solution is to introduce a trusted central authority, or mint, that checks every transaction for double spending., After each transaction, the coin must be returned to the mint to issue a new coin, and only coins issued directly from the mint are trusted not to be double-spent., The problem with this solution is that the fate of the entire money system depends on the company running the mint, with every transaction having to go through them, just like a bank., This adds an incentive for nodes to support the network, and provides\na way to initially distribute coins into circulation, since there is no central authority to issue them., The steady addition of a constant of amount of new coins is analogous to gold miners expending\nresources to add gold to circulation., Once a predetermined number of coins have entered\ncirculation, the incentive can transition entirely to transaction fees and be completely inflation\nfree., If a greedy attacker is able to\nassemble more CPU power than all the honest nodes, he would have to choose between using it\nto defraud people by stealing back his payments, or using it to generate new coins., He ought to\nfind it more profitable to play by the rules, such rules that favour him with more new coins than\neveryone else combined, than to undermine the system and the validity of his own wealth., Although it would be possible to handle coins individually, it would be unwieldy to make a separate transaction for every cent in a transfer., Even if this is accomplished, it does not throw the system open to arbitrary changes, such as creating value out of thin air or taking money that never belonged to the attacker., An attacker can only try to change one of his own transactions to take back money he recently spent., We assume the sender is an attacker who wants to make the recipient believe he paid him for a while, then switch it to pay back to himself after some time has passed.]"
